In [ ]:
import sys, os
import time
import cv2

from makeDiambraEnv import *

timeDepSeed = int((time.time()-int(time.time()-0.5))*1000)

In [ ]:
base_path = "/home/yourUsername/DIAMBRAenvironment/"

diambraEnvKwargs = {}
diambraEnvKwargs["gameId"]          = "doapp"
diambraEnvKwargs["diambraEnv_path"] = base_path + "diambraEnvLib/"
diambraEnvKwargs["roms_path"]       = base_path + "roms/" # Absolute path to roms
diambraEnvKwargs["mame_path"]       = base_path + "mame/" # Absolute path to MAME executable

diambraEnvKwargs["mame_diambra_step_ratio"] = 6
diambraEnvKwargs["render"]      = True
diambraEnvKwargs["lock_fps"]    = True # Locks to 60 FPS
diambraEnvKwargs["sound"]       = diambraEnvKwargs["lock_fps"] and diambraEnvKwargs["render"]

# 1P
diambraEnvKwargs["player"] = "Random"
# 2P
#diambraEnvKwargs["player"] = "P1P2"

# Game specific
diambraEnvKwargs["difficulty"] = 3
diambraEnvKwargs["characters"]  = [["Random", "Random"], ["Random", "Random"]]
diambraEnvKwargs["charOutfits"] = [2, 2]

In [ ]:
# DIAMBRA gym kwargs
diambraGymKwargs = {}
diambraGymKwargs["P2brain"]               = None#gamePad_policy
diambraGymKwargs["continue_game"]         = 0.0
diambraGymKwargs["show_final"]            = False
diambraGymKwargs["gamePads"]              = [None, diambraGymKwargs["P2brain"]]
diambraGymKwargs["actionSpace"]           = ["discrete", "multiDiscrete"]
diambraGymKwargs["attackButCombinations"] = [True, False]
diambraGymKwargs["actBufLen"]             = 12

In [ ]:
# Wrappers kwargs
wrapperKwargs = {}
wrapperKwargs["hwc_obs_resize"]    = [256, 256, 1]
wrapperKwargs["normalize_rewards"] = True
wrapperKwargs["clip_rewards"]      = False
wrapperKwargs["frame_stack"]       = 6
wrapperKwargs["dilation"]          = 1
wrapperKwargs["scale"]             = True
wrapperKwargs["scale_mod"]         = 0

In [ ]:
# Additional Observations

#keyToAdd = None

keyToAdd = []
keyToAdd.append("ownActionsBuf")
if diambraEnvKwargs["player"] == "P1P2":
    keyToAdd.append("oppActionsBuf") # Only 2P

if diambraEnvKwargs["gameId"] != "tektagt": # DOA, SFIII, UMK3
    keyToAdd.append("ownHealth")
    keyToAdd.append("oppHealth")
else: # TEKTAG
    keyToAdd.append("ownHealth_1")
    keyToAdd.append("ownHealth_2")
    keyToAdd.append("oppHealth_1")
    keyToAdd.append("oppHealth_2")

keyToAdd.append("ownPosition")
keyToAdd.append("oppPosition")
#keyToAdd.append("ownWins")
#keyToAdd.append("oppWins")
keyToAdd.append("stage")
keyToAdd.append("characters")

In [ ]:
envId = "Test"
env = make_diambra_env(diambraGym, env_prefix=envId, seed=timeDepSeed, 
                       diambra_kwargs=diambraEnvKwargs, 
                       diambra_gym_kwargs=diambraGymKwargs,
                       wrapper_kwargs=wrapperKwargs, 
                       key_to_add=keyToAdd)

In [ ]:
print("Obs space =", env.observation_space)
print("Obs space type =", env.observation_space.dtype)
print("Obs space high bound =", env.observation_space.high)
print("Obs space low bound =", env.observation_space.low)

In [ ]:
# Printing action spaces
for idx in range(2):
    
    if diambraEnvKwargs["player"] != "P1P2" and idx == 1:
        continue
        
    print("Action space {}P = ".format(idx+1), env.action_spaces[idx])
    print("Action space type P{} = ".format(idx+1), env.action_spaces[idx].dtype)
    if diambraGymKwargs["actionSpace"][idx] == "multiDiscrete":
        print("Action space n = ", env.action_spaces[idx].nvec)
    else:
        print("Action space n = ", env.action_spaces[idx].n)

In [ ]:
limAct = [None, None]
for idx in range(2):
    limAct[idx] = [env.actBufLen * env.n_actions[idx][0], 
                   env.actBufLen * env.n_actions[idx][0] + env.actBufLen * env.n_actions[idx][1]]

In [ ]:
# Visualize Obs content
def observationViz(observation, limAct):
    
        shp = observation.shape
        additionalPar = int(observation[0,0,shp[2]-1])
        
        nScalarAddPar = additionalPar - 2*len(env.charNames)\
                        - env.actBufLen*(env.n_actions[0][0]+env.n_actions[0][1]) # 1P
        
        if diambraEnvKwargs["player"] == "P1P2": 
            nScalarAddPar = additionalPar - 2*len(env.charNames)\
                            - env.actBufLen*(env.n_actions[0][0]+env.n_actions[0][1]+\
                                             env.n_actions[1][0]+env.n_actions[1][1])# 2P
        
        addPar = observation[:,:,shp[2]-1]
        addPar = np.reshape(addPar, (-1))
        addPar = addPar[1:additionalPar+1]
        actions = addPar[0:additionalPar-nScalarAddPar-2*env.numberOfCharacters]
        
        moveActionsP1   = actions[0:limAct[0][0]]
        attackActionsP1 = actions[limAct[0][0]:limAct[0][1]]
        moveActionsP1   = np.reshape(moveActionsP1, (env.actBufLen,-1))
        attackActionsP1 = np.reshape(attackActionsP1, (env.actBufLen,-1))
        print("Move actions P1 =\n", moveActionsP1)
        print("Attack actions P1 =\n ", attackActionsP1)
        
        # 2P
        if diambraEnvKwargs["player"] == "P1P2":
            moveActionsP2   = actions[limAct[0][1]:limAct[0][1]+limAct[1][0]]
            attackActionsP2 = actions[limAct[0][1]+limAct[1][0]:limAct[0][1]+limAct[1][1]]
            moveActionsP2   = np.reshape(moveActionsP2, (env.actBufLen,-1))
            attackActionsP2 = np.reshape(attackActionsP2, (env.actBufLen,-1))
            print("Move actions P2 =\n", moveActionsP2)
            print("Attack actions P2 =\n", attackActionsP2)        
        
        others = addPar[additionalPar-nScalarAddPar-2*env.numberOfCharacters:]
        print("ownHealth = ", others[0])
        print("oppHealth = ", others[1])
        print("ownPosition = ", others[2])
        print("oppPosition = ", others[3])
        print("stage = ", others[4])
        print("Playing Char 1P = ", env.charNames[list(others[nScalarAddPar:
                                                              nScalarAddPar + env.numberOfCharacters]).index(1.0)])
        
        if diambraEnvKwargs["player"] == "P1P2":
            print("Playing Char 2P = ", env.charNames[list(others[nScalarAddPar + env.numberOfCharacters:
                                                                  nScalarAddPar + 2*env.numberOfCharacters]).index(1.0)])
        
        #input("Pause1")
        
        
        obs = np.array(observation).astype(np.float32)
    
        for idx in range(shp[2]-1):
            cv2.imshow("image"+str(idx), obs[:,:,idx])
    
        cv2.waitKey()

In [ ]:
observation = env.reset()

In [ ]:
observationViz(observation, limAct) # Press space bar to complete notebook cell execution

In [ ]:
cumulativeEpRew = 0.0
cumulativeEpRewAll = []

maxNumEp = 100
currNumEp = 0

while currNumEp < maxNumEp:

    # 1P
    action = env.action_spaces[0].sample()
    
    # 2P
    action2 = env.action_spaces[1].sample()
    if diambraEnvKwargs["player"] == "P1P2":
        action = np.append(action, action2)

    print("Action:", action)
    observation, reward, done, info = env.step(action)
    
    # Visualize observations content
    observationViz(observation, limAct) # Keep space bar pressed to continue env execution
        
    cumulativeEpRew += reward
    
    if np.any(done):
        currNumEp += 1
        print("Ep. # = ", currNumEp)
        print("Ep. Cumulative Rew # = ", cumulativeEpRew)
        cumulativeEpRewAll.append(cumulativeEpRew)
        cumulativeEpRew = 0.0

        observation = env.reset()

print("Mean cumulative reward = ", np.mean(cumulativeEpRewAll))    
print("Std cumulative reward = ", np.std(cumulativeEpRewAll))       
    
env.close()